Business Problem: Predict total price of a trip by considering different parameters .

In [ ]:
import pandas as pd
from google.colab import files
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV



In [ ]:

uploaded=files.upload()

Saving filtered_data_month_1 (1).csv to filtered_data_month_1 (1).csv


cleaning the dataset

In [ ]:


df=pd.read_csv("filtered_data_month_1 (1).csv")
print(df.info())
correlation_matrix=df.corr()
print(correlation_matrix)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17536 entries, 0 to 17535
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   trip_distance        17536 non-null  float64
 1   rate_code            17536 non-null  int64  
 2   payment_type         17536 non-null  int64  
 3   fare_amount          17536 non-null  float64
 4   extra                17536 non-null  float64
 5   mta_tax              17536 non-null  float64
 6   tip_amount           17536 non-null  float64
 7   tolls_amount         17536 non-null  float64
 8   imp_surcharge        17536 non-null  float64
 9   total_amount         17536 non-null  float64
 10  pickup_location_id   17536 non-null  int64  
 11  dropoff_location_id  17536 non-null  int64  
 12  hour_of_day          17536 non-null  int64  
 13  trip_duration        17536 non-null  float64
dtypes: float64(9), int64(5)
memory usage: 1.9 MB
None
                     trip_distance  

his code evaluates a linear regression model on a dataset with total_amount as the target variable. The steps are as follows:

Data Splitting: The dataset is split into training and test sets using an 80-20 ratio.
Cross-Validation: The model is trained using 5-fold cross-validation on the training data, with Mean Squared Error (MSE) and R-squared scores computed for each fold.
Model Training and Prediction: The model is then trained on the full training set and used to make predictions on the test set.
Performance Evaluation: The MSE and R-squared scores are calculated for the test set, and a DataFrame is created showing the actual and predicted values.

In [ ]:


# Prepare the data
X = df.drop('total_amount', axis=1)
y = df['total_amount']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Linear Regression model
model = LinearRegression()

# Perform 5-fold cross-validation and get MSE for each fold
mse_scores = -cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
r2_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')

# Print cross-validated MSE and R-squared scores for each fold
print("Cross-validated Mean Squared Errors:", mse_scores)
print("Average Cross-validated MSE:", np.mean(mse_scores))
print("Cross-validated R-squared Scores:", r2_scores)
print("Average Cross-validated R-squared:", np.mean(r2_scores))

# Fit model and predict for comparison with test data
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Print actual vs predicted for test set
rdf1 = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(rdf1)

# Calculate and print MSE and R-squared for test set
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error on Test Set:", mse)
print("R-squared on Test Set:", r2)


Cross-validated Mean Squared Errors: [0.03725252 0.03154283 0.03607073 0.03545853 0.04797708]
Average Cross-validated MSE: 0.03766033808131891
Cross-validated R-squared Scores: [0.99990196 0.99990539 0.99990495 0.99991051 0.99988933]
Average Cross-validated R-squared: 0.9999024281591835
       Actual  Predicted
3784    32.80  32.787685
2175    37.85  37.886225
9253    39.10  39.117526
15869   42.07  42.083724
4201    30.30  30.337533
...       ...        ...
17382   24.30  24.303963
9595    27.80  27.792096
12371   85.85  85.859629
14208   38.74  38.774051
7800    30.30  30.337898

[3508 rows x 2 columns]
Mean Squared Error on Test Set: 0.02468030028495565
R-squared on Test Set: 0.9999371147721693


Results Interpretation
Cross-validated Mean Squared Errors: The MSE values across the 5 folds are all low, with an average MSE of approximately 0.0377. This indicates that the model's errors are small, suggesting it generalizes well on unseen data.
Cross-validated R-squared Scores: The average R-squared score is approximately 0.9999, which suggests an excellent fit on the training data, as R-squared values close to 1 indicate the model explains almost all of the variance in the target variable.
Test Set Results: On the test set, the model’s MSE is 0.0247, and the R-squared is 0.9999, which are consistent with the cross-validation scores. This consistency implies that the model performs well both in cross-validation and on the test data, indicating minimal overfitting.

This code utilizes a Decision Tree Regressor for predicting the total_amount in the dataset. The steps are as follows:

Data Splitting: The dataset is divided into training and testing sets, with 80% for training and 20% for testing.
Randomized Parameter Search: A parameter grid is defined for the decision tree, including values for max_depth, min_samples_split, min_samples_leaf, and max_features. RandomizedSearchCV is used to find the best combination of these hyperparameters by performing 5-fold cross-validation on 10 randomly selected parameter configurations.
Model Training and Prediction: The model with the best parameters is chosen and trained on the training set. Predictions are then made on the test set.
Results Display and Evaluation: The actual vs. predicted values for the test set are displayed in a DataFrame. The Mean Squared Error (MSE) and R-squared score are calculated to evaluate model performance.

In [ ]:


# Prepare the data
X = df.drop('total_amount', axis=1)
y = df['total_amount']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Decision Tree Regressor
dtr = DecisionTreeRegressor(random_state=42)

# Define the parameter grid for randomized search
param_dist = {
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': np.arange(2, 21, 2),
    'min_samples_leaf': np.arange(1, 21, 2),
    'max_features': [None,'sqrt', 'log2']
}

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    dtr, param_distributions=param_dist, n_iter=10, scoring='neg_mean_squared_error', cv=5, random_state=42
)

# Fit the randomized search model
random_search.fit(X_train, y_train)

# Get the best model from random search
best_model = random_search.best_estimator_

# Predict on the test set using the best model
y_pred = best_model.predict(X_test)

# Create a DataFrame to compare actual and predicted values
rdf = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(rdf)

# Calculate and print the evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Best Parameters:", random_search.best_params_)
print("Mean Squared Error:", mse)
print("R-squared:", r2)


       Actual  Predicted
3784    32.80  32.998421
2175    37.85  37.986970
9253    39.10  38.990690
15869   42.07  42.070000
4201    30.30  30.197368
...       ...        ...
17382   24.30  24.300000
9595    27.80  28.014286
12371   85.85  90.242432
14208   38.74  38.304571
7800    30.30  29.262963

[3508 rows x 2 columns]
Best Parameters: {'min_samples_split': 10, 'min_samples_leaf': 19, 'max_features': None, 'max_depth': 40}
Mean Squared Error: 7.478921787450236
R-squared: 0.9809437610117511


The Linear Regression model performs better than the Decision Tree Regressor in both MSE and R-squared metrics, indicating that it may be more suitable for this dataset, especially if the target variable has a linear relationship with the features. The Decision Tree, while effective, might be overfitting slightly or less suited to capture the precise linear pattern indicated by the R-squared results.

the code aims to identify the best Random Forest model for predicting total_amount by testing different parameter combinations and evaluating performance using MSE and R-squared metrics.

In [ ]:


# Define the parameter grid for randomized search
param_dist = {
    'n_estimators': [50, 100, 200, 300, 400],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': np.arange(2, 21, 2),
    'min_samples_leaf': np.arange(1, 21, 2),
    'max_features': [ 'sqrt', 'log2']
}

# Initialize RandomForestRegressor
rf = RandomForestRegressor(random_state=42)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    rf, param_distributions=param_dist, n_iter=10, scoring='neg_mean_squared_error',
    cv=5, random_state=42, n_jobs=-1
)

# Fit RandomizedSearchCV
random_search.fit(X_train, y_train)

# Get the best estimator
best_rf = random_search.best_estimator_

# Predict with the best model
y_pred = best_rf.predict(X_test)

# Create DataFrame with Actual and Predicted values
rdf3 = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(rdf3)

# Calculate Mean Squared Error and R-squared
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R-squared:", r2)


       Actual  Predicted
3784    32.80  33.542050
2175    37.85  38.371192
9253    39.10  40.180170
15869   42.07  42.278580
4201    30.30  30.267229
...       ...        ...
17382   24.30  24.288926
9595    27.80  28.953414
12371   85.85  88.340121
14208   38.74  37.918810
7800    30.30  30.354171

[3508 rows x 2 columns]
Mean Squared Error: 7.520963093005291
R-squared: 0.980836640067208


Mean Squared Error (MSE):

The Linear Regression model outperformed both the Decision Tree and Random Forest models by achieving a significantly lower MSE (0.0247) compared to the Random Forest's 7.52. This suggests that Linear Regression’s predictions are closest to the actual values among the three models.
The Random Forest and Decision Tree models have similar MSEs, with the Decision Tree being slightly better by a small margin.
R-squared Score:

The R-squared score for Linear Regression (0.9999) indicates an almost perfect fit, meaning it can explain nearly all the variance in the target variable. This R-squared score is considerably higher than both the Decision Tree (0.9810) and Random Forest (0.9808) models.
The Random Forest and Decision Tree models have nearly identical R-squared scores, suggesting that they capture approximately 98% of the variance in the target variable, although they are both outperformed by Linear Regression.
Conclusion:
The Linear Regression model remains the best-performing model on this dataset, both in terms of minimizing error (MSE) and maximizing explained variance (R-squared). While the Random Forest model often reduces overfitting compared to single trees, here it performs similarly to the Decision Tree Regressor, possibly due to the data structure aligning well with a linear relationship. This suggests that Linear Regression may be most appropriate if the relationship in the data is indeed linear.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
# Define parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 1],
    'kernel': ['linear', 'poly', 'rbf']
}

# Initialize SVR model
svr = SVR()

# Set up GridSearchCV
grid_search = GridSearchCV(svr, param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

# Output the best parameters
print("Best Parameters:", grid_search.best_params_)
